# Changer le CSV et le nom des DF

- output_(n).csv
cellule 4

- df_data_villes_clean(n)
cellules: 4, 5, 6

- df_meilleur_agent(n)
cellules: 7, 8

# Vérifier la shape de df_data_villes_clean(n) et df_meilleur_agent(n) 

il doit y avoir le même nombre de lignes

In [ ]:
#pip install selenium selenium-stealth beautifulsoup4 pandas

In [ ]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import re, time, random


In [ ]:
df_data_villes_clean11 = pd.read_csv("output_11.csv", 
                                    index_col=0,
                                    dtype=str)


In [ ]:
df_data_villes_clean11.head()

In [ ]:
def extract_number(text):
    """Nettoyer le texte et transformer en float"""
    clean_text = text.replace("\u202f", "").replace("\xa0", "").replace("€", "").strip()
    clean_text = clean_text.replace(",", ".")
    return float(clean_text)

# Selenium setup (Chrome auto-managed by Selenium Manager; Chrome must be installed)
options = webdriver.ChromeOptions()
options.add_argument("--lang=fr-FR")
options.add_argument("--disable-blink-features=AutomationControlled")
# Uncomment to persist cookies/session between runs:
# options.add_argument(r"--user-data-dir=C:\Users\Dell\AppData\Local\Google\Chrome\User Data\ScrapeProfile")

driver = webdriver.Chrome(options=options)
stealth(driver,
    languages=["fr-FR", "fr"],
    vendor="Google Inc.",
    platform="Win32",
    webgl_vendor="Intel Inc.",
    renderer="Intel Iris OpenGL Engine",
    fix_hairline=True,
)
driver.set_window_size(1366, 768)

# Lancer le navigateur
driver.get("https://www.meilleursagents.com/prix-immobilier/")

In [ ]:
# Liste pour stocker toutes les données
all_data = []
error_data = []

for index, row in df_data_villes_clean11.iterrows():
    city = row['Lib_MA']+" ("+row['Code_postal']+")"
    try:
        # Human-like pacing
        time.sleep(random.uniform(1.8, 3.6))

        # Trouver la barre de recherche
        search_box = driver.find_element(By.NAME, "q")

        # Simuler une frappe clavier, caractère par caractère
        for y in city:
            search_box.clear()
            search_box.send_keys(y)
            time.sleep(0.05)  # petite pause pour rendre la frappe réaliste

        # Valider avec la touche Entrée
        time.sleep(1)
        search_box.send_keys(Keys.RETURN)
        time.sleep(2)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        raw_text3 = soup.find_all("ul", class_="prices-summary__price-range")
        data_dict = {}
        if not raw_text3 :
            driver.refresh()
            time.sleep(3)
        else:
            # Ajouter la ville
            title_text = soup.find("title").get_text().strip()
            ville = title_text.split("Prix immobilier")[1].split("(")[0].strip()
            data_dict["ville"] = ville

            for ul in raw_text3:
                li_items = ul.find_all("li")
                label = li_items[0].get_text().strip().lower()
                main_value = extract_number(li_items[1].get_text())

                # Nettoyer le texte pour récupérer min et max correctement
                range_text = li_items[2].get_text()
                range_text = range_text.replace("\u202f", "").replace("\xa0", "").replace(",", ".")

                # Extraire tous les nombres sous forme de float
                numbers_in_range = [float(n) for n in re.findall(r"[\d\.]+", range_text)]
                if "prix" in label:
                    if "prix_appartement" not in data_dict:
                        data_dict["prix_appartement"] = main_value
                        data_dict["min_appartement"] = numbers_in_range[0]
                        data_dict["max_appartement"] = numbers_in_range[1]
                    else:
                        data_dict["prix_maison"] = main_value
                        data_dict["min_maison"] = numbers_in_range[0]
                        data_dict["max_maison"] = numbers_in_range[1]
                elif "loyer" in label:
                    if "loyer_appartement" not in data_dict:
                        data_dict["loyer_appartement"] = main_value
                        data_dict["loyer_min_appartement"] = numbers_in_range[0]
                        data_dict["loyer_max_appartement"] = numbers_in_range[1]
                    else:
                        data_dict["loyer_maison"] = main_value
                        data_dict["loyer_min_maison"] = numbers_in_range[0]
                        data_dict["loyer_max_maison"] = numbers_in_range[1]

            # Ajouter l'URL pour référence
            data_dict["city"] = city
            all_data.append(data_dict)
            print(f"Datas ajoutées pour la ville : {ville}")

    except Exception as e:
        print(f"Erreur pour la ville {city}: {e}")
        error_data.append({
        "city": city,
        "erreur": str(e)})

    # Pause pour ne pas surcharger le site
    time.sleep(random.uniform(1.8, 3.6))

driver.quit()

# Convertir en DataFrame
# df_scraped = pd.DataFrame(all_data)
# df_scraped.head()

In [ ]:
driver.quit()


In [ ]:
df_meilleur_agent11 = pd.DataFrame(all_data)
df_error_ma_11 = pd.DataFrame(error_data)

In [ ]:
df_meilleur_agent11.to_csv('ma_11.csv',index=False)
df_error_ma_11.to_csv('error_ma_11.csv',index=False)

In [ ]:
df_meilleur_agent11.shape
df_error_ma_11.shape

In [ ]:
df_ma11.head()